In [1]:
import os
import numpy as np
import pandas as pd

In [3]:
root = os.path.join(os.getcwd(), 'DATA')
df = pd.read_csv(os.path.join(root, 'data_for_dash.csv'), index_col = 0)

In [14]:
# Set Jikwon info
selected_jikwon = 6163718
data = df[df['JIKWON_NO'] == selected_jikwon]
selected_name = data['NAME'].unique()[0].rstrip()

# create nodes
elements=[]
# -- Jikwon Node
elements.append({'data': {'id': selected_name, 'label': selected_name},
                'position': {'x': 150, 'y': 150},
                'grabbable': False})
# -- Program Nodes
for name in list(data['A'].unique()):
    elements.append({'data': {'id': name, 'label': name, 'size': data[data["A"] == name]["A"].count()}})

for name in list(data['프로그램종류'].unique()):
    elements.append({'data': {'id': name, 'label': name, 'size': data[data["프로그램종류"] == name]["프로그램종류"].count()},
                     'classes': 'notRoot'})

# create edges
for t in list(data['A'].unique()):
    elements.append({'data': {'source': selected_name, 'target': t}})

for f, t in zip(list(data['A']), list(data['프로그램종류'])):
    elements.append({'data': {'source': f, 'target': t}})

In [10]:
name = "C"
data[data["A"] == name]["A"].count()

154

In [11]:
data[data["A"] == name]

,JIKWON_NO,시스템유형,업무그룹,업무,프로그램종류,A,A_COUNT,B_COUNT,NAME
293422,6163718,코어,External Channel,펌뱅킹,배치,C,5398,531,권태원
293423,6163718,코어,External Channel,펌뱅킹,온라인서비스,C,15617,531,권태원
293425,6163718,단위,단위-대외계,단위-대외계펌뱅킹,MODULE,C,922,221,권태원
293427,6163718,코어,External Channel,펌뱅킹,거래코드,C,6356,531,권태원
293428,6163718,코어,External Channel,펌뱅킹,온라인서비스,C,15617,531,권태원
...,...,...,...,...,...,...,...,...,...
293703,6163718,코어,External Channel,펌뱅킹,온라인서비스,C,15617,531,권태원
293704,6163718,단위,단위-대외계,단위-대외계펌뱅킹,온라인서비스,C,15617,221,권태원
293705,6163718,단위,단위-대외계,단위-대외계펌뱅킹,온라인서비스,C,15617,221,권태원
293706,6163718,코어,External Channel,펌뱅킹,온라인서비스,C,15617,531,권태원


In [15]:
elements

[{'data': {'id': '권태원', 'label': '권태원'},
  'position': {'x': 150, 'y': 150},
  'grabbable': False},
 {'data': {'id': 'C', 'label': 'C', 'size': 154}},
 {'data': {'id': '프로프레임', 'label': '프로프레임', 'size': 50}},
 {'data': {'id': 'xml', 'label': 'xml', 'size': 9}},
 {'data': {'id': 'HEADER', 'label': 'HEADER', 'size': 3}},
 {'data': {'id': 'Web', 'label': 'Web', 'size': 7}},
 {'data': {'id': 'iDev@Tool', 'label': 'iDev@Tool', 'size': 33}},
 {'data': {'id': 'iDev@BuildLink', 'label': 'iDev@BuildLink', 'size': 22}},
 {'data': {'id': 'Shell', 'label': 'Shell', 'size': 9}},
 {'data': {'id': '배치', 'label': '배치', 'size': 17}, 'classes': 'notRoot'},
 {'data': {'id': '온라인서비스', 'label': '온라인서비스', 'size': 70},
  'classes': 'notRoot'},
 {'data': {'id': 'DBIO', 'label': 'DBIO', 'size': 10}, 'classes': 'notRoot'},
 {'data': {'id': 'MODULE', 'label': 'MODULE', 'size': 19},
  'classes': 'notRoot'},
 {'data': {'id': 'XML', 'label': 'XML', 'size': 9}, 'classes': 'notRoot'},
 {'data': {'id': '거래코드', 'label'

In [ ]:
# -*- coding: utf-8 -*-

# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

# Import Libraries
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_cytoscape as cyto
from dash.dependencies import Input, Output
import plotly.express as px
import pandas as pd
import os

# ################################ STYLESHEET ################################   
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets, suppress_callback_exceptions=True)

# ################################### TAB1 ###################################    
# Get data
root = os.path.join(os.getcwd(), 'DATA')
df = pd.read_csv(os.path.join(root, 'data_for_dash.csv'), index_col = 0)
elements, stylesheet = [], []

# -------------------------------- STYLESHEET --------------------------------
# -- Group selector
stylesheet.append({'selector': 'node',
                   'style': {'content': 'data(label)'}
                  })
# -- Class selector
stylesheet.append({'selector': '.root',
                   'style': {'background-color': 'blue'}
                  })
stylesheet.append({'selector': '.A',
                   'style': {'width': 'data(size)',
                             'height': 'data(size)'}
                  })
stylesheet.append({'selector': '.program',
                   'style': {'width': 'data(size)',
                             'height': 'data(size)'}
                  })


# --------------------------------- CALLBACKS --------------------------------    
@app.callback(
    dash.dependencies.Output('cytoscape', 'elements'),
    [dash.dependencies.Input('btn', 'n_clicks')],
    [dash.dependencies.State('ibx', 'value')])
def update_output(n_clicks, value):
    if value == "" or int(value) not in (df['JIKWON_NO'].unique()):  # 검색한 직원이 없거나 존재하지 않는 경우
        return []
    
    # Set Jikwon info
    selected_jikwon = int(value)
    data = df[df['JIKWON_NO'] == selected_jikwon]
    selected_name = data['NAME'].unique()[0].rstrip()

    # create nodes
    elements=[]
    # -- Jikwon Node
    elements.append({'data': {'id': selected_name, 'label': selected_name},
                     'classes': 'root',
                     'position': {'x': 150, 'y': 150},
                     'grabbable': False})
    # -- Program Nodes
    for name in list(data['A'].unique()):
        elements.append({'data': {'id': name, 'label': name, 'size': data[data["A"] == name]["A"].count()},
                         'classes': 'A'})

    for name in list(data['프로그램종류'].unique()):
        elements.append({'data': {'id': name, 'label': name, 'size': data[data["프로그램종류"] == name]["프로그램종류"].count()},
                         'classes': 'program'})

    # create edges
    for t in list(data['A'].unique()):
        elements.append({'data': {'source': selected_name, 'target': t}})

    for f, t in zip(list(data['A']), list(data['프로그램종류'])):
        elements.append({'data': {'source': f, 'target': t}})

    return elements

# ################################### TAB2 ###################################    

# --------------------------------- CALLBACKS --------------------------------    


# ################################# LAYOUT ###################################    
app.layout = html.Div(children=[
    html.H1(children='Data Driven HRM',
            style={'textAlign': 'center'}),
    
    html.Div(className='eight columns', children=[cyto.Cytoscape(
        id='cytoscape',
        elements=elements,
        layout={'name': 'cose'},
        style={'width': '1100px', 'height': '500px'},
        stylesheet=stylesheet)
    ]),
    
    html.Div(className='four columns', children=[
        dcc.Tabs(id='tabs', children=[
            dcc.Tab(label='기술 역량 네트워크', children=[
                dcc.Input(
                    id='ibx',
                    placeholder='직원번호 검색',
                    type='text',
                    value=''
                ),
                html.Button('조회', id='btn'),
            ]),

            dcc.Tab(label='업무 역량 네트워크', children=[
                ])
            ])
        ]),

    ]),
    
    html.Div(id='tabs-content'), 
])

# --------------------------------- CALLBACKS --------------------------------    
@app.callback(Output('tabs-content', 'children'),
              Input('tabs', 'value'))
def render_content(tab):
    if tab == 'tab-1':
        return html.Div([
                cyto.Cytoscape(id='cytoscape',
                               elements=elements,
                               layout={'name': 'cose'},
                               style={'width': '1100px', 'height': '500px'},
                               stylesheet=stylesheet)
        ])
    elif tab == 'tab-2':
        return html.Div([
            html.H3('Tab content 2')
        ])

if __name__ == '__main__':
    app.run_server(debug=True)  # automatically refresh page when code is changed